In [1]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from random import sample

Using TensorFlow backend.


### Load Vocab

In [3]:
vocab = {}
idToWord = {}
with open('vocab250.txt') as f:
    words = f.read().splitlines()
    for wordIndex in words:
        word, index = wordIndex.split(' -----> ')
        vocab[word] = index
        idToWord[int(index)] = word

In [4]:
vocab['<pad_token>']

'251'

### Create input squences

In [7]:
look_back_len = 100 + 1
sequences = []
vocabulary_size = len(vocab)

with open('int_seq_250_vocab.txt') as f:
    files = f.read().splitlines()
    for file in files:
        numbers = list(map(int, file.split(',')[:-1]))
        for i in range(look_back_len, len(numbers)):
            seq = numbers[i-look_back_len:i]
            sequences.append(seq)
    sequence_small = sample(sequences, int(len(sequences)))

    n_sequences = np.empty([len(sequence_small), look_back_len], dtype='int32')
    for i in range(len(sequence_small)):
        n_sequences[i] = sequences[i]

print(n_sequences.shape)

(9824993, 101)


### Convert output to one hot encoded vector

In [8]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
print(len(train_targets))

train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_targets[0])

9824993
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Train Model

In [9]:
import keras
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.save("model_250Vocab{}.hd5".format(epoch))

In [10]:
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

saver = CustomSaver()

model.fit(train_inputs,train_targets, epochs = 20 ,verbose=1, validation_split=0.3, callbacks=[saver])
model.save("mymodel-250Vocab.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          25200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 50)           30200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 252)               12852     
Total params: 91,002
Trainable params: 91,002
Non-trainable params: 0
_________________________________________________________________
None


c:\Miniconda\envs\azureml_py36_tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6877495 samples, validate on 2947498 samples
Epoch 1/20
6877495/6877495 [==============================] - 22898s 3ms/step - loss: 1.2231 - accuracy: 0.6666 - val_loss: 1.2076 - val_accuracy: 0.6632
Epoch 2/20
  92672/6877495 [..............................] - ETA: 5:36:44 - loss: 1.0965 - accuracy: 0.6889

KeyboardInterrupt: 

### Test Model

In [36]:
from keras.preprocessing.sequence import pad_sequences

# sample python input
"""
num = 46
print
"""
encoded_text = [2,13,35,5]
seq_len = 6
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre', value = int(vocab['<pad_token>']))
print(encoded_text, pad_encoded[0])

print("Top 3 Suggestions:")
#for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
#    print(idToWord[i])

[2, 13, 35, 5] [1001 1001    2   13   35    5]
Top 3 Suggestions:
